In [1]:
# Tutorial 3: More Networks and More Areas
# by Yu Wang

In [2]:
# download data from kaggle
# https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/

! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle datasets download sdss-iii-iv -f dr16q-log-zvi-galaxy.h5
! kaggle datasets download sdss-iii-iv -f dr16q-log-zvi-quasar.h5

! unzip dr16q-log-zvi-galaxy.h5.zip
! unzip dr16q-log-zvi-quasar.h5.zip

 99% 921M/934M [00:08<00:00, 122MB/s]
100% 934M/934M [00:08<00:00, 118MB/s]
100% 11.8G/11.8G [04:17<00:00, 83.7MB/s]
100% 11.8G/11.8G [04:17<00:00, 49.4MB/s]
Archive:  dr16q-log-zvi-galaxy.h5.zip
  inflating: dr16q-log-zvi-galaxy.h5  
Archive:  dr16q-log-zvi-quasar.h5.zip
  inflating: dr16q-log-zvi-quasar.h5  


In [5]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import time
import os
import h5py
import sys
#sys.path.append('../')
#from file_path import *

import torchvision
from torch.utils.data import Dataset, DataLoader, TensorDataset, ConcatDataset, random_split
from torchvision import transforms
from torch.utils import data
from sklearn.model_selection import KFold, train_test_split
#from skorch.dataset import Dataset

#import seaborn as sns
#sns.set_theme(style="ticks")

In [6]:
# load h5 data
galaxy = h5py.File('./dr16q-log-zvi-galaxy.h5', 'r') 
quasar = h5py.File('./dr16q-log-zvi-quasar.h5', 'r') 

quasar.keys()

<KeysViewHDF5 ['autoclass_dr14q', 'autoclass_pqn', 'bal_prob', 'class_person', 'deltachi2_pca', 'fiberid', 'flux', 'flux_norm', 'index', 'is_qso_dr12q', 'is_qso_final', 'is_qso_qn', 'mjd', 'plate', 'sdss_name', 'sn_median_all', 'thing_id', 'url', 'wavelength', 'wavelength_log', 'z', 'z_conf', 'z_dr12q', 'z_pca', 'z_pipe', 'z_qn', 'z_vi', 'zwarn_pca']>

In [ ]:
# use all data
# set same number of each class
#feature = np.concatenate((galaxy['flux_norm'], quasar['flux_norm'], bal['flux_norm']))
#label = np.concatenate((np.full((len(galaxy['flux_norm']), 1), 0), np.full((len(quasar['flux_norm']), 1), 1), np.full((len(bal['flux_norm']), 1), 2)))

In [7]:
# set same number of each class

#number = min(len(galaxy['plate']), len(quasar['plate']))
#print(number)

number = 3000 # for having a quick traning in the workshop, we adopt 3000 spetra of each class
feature = np.concatenate((galaxy['flux_norm'][:number], quasar['flux_norm'][:number]))
label = np.concatenate((np.full((number, 1), 0), np.full((number, 1), 1)))

In [8]:
# split traning and test data
X_train, X_test, Y_train, Y_test = train_test_split(feature ,label, test_size= 0.2,random_state = 42 )

In [9]:
X_train=torch.Tensor(X_train).unsqueeze(1)
Y_train=torch.Tensor(Y_train).to(torch.long)

X_test=torch.Tensor(X_test).unsqueeze(1)
Y_test=torch.Tensor(Y_test).to(torch.long)

## FNet

In [ ]:
class FNet(nn.Module):
    def __init__(self):
        super(FNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(1, 60, 200),
            nn.ReLU(),
            nn.MaxPool1d(2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv1d(60, 70,200),
            nn.ReLU(),
            nn.MaxPool1d(2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv1d(70, 36, 32),
            nn.ReLU(),
            nn.MaxPool1d(2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Sequential(
            nn.Linear(17532, 900),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Linear(900, 100),
            nn.ReLU())
        self.fc3 = nn.Linear(100, 2)

    def forward(self, x):
        #x = x.unsqueeze(1)
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [ ]:
net= FNet()

In [ ]:
## test
x = torch.rand(1,1,4618)
net(x)

tensor([[0.0833, 0.0372]], grad_fn=<AddmmBackward>)

## ResNet

In [ ]:
class Residual(nn.Module):
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=201, padding=100, stride=stride)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=17, padding=8, stride=1)
        if use_1x1conv:
            self.conv3 = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)
        #self.fc = nn.Linear(in_features, out_features, bias=True)
        
    def forward(self, x):
        y1 = F.relu(self.bn1(self.conv1(x)))
        y2 = self.bn2(self.conv2(y1))
        if self.conv3:
            x = self.conv3(x)
        return F.relu(y2+x)
    
    
def resnet_block(in_channels, out_channels, num_residuals, stride=1):
    blk = []
    for i in range(num_residuals):
        if i==0:
            blk.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=stride))
        else:
            blk.append(Residual(out_channels, out_channels))
    return nn.Sequential(*blk)

In [ ]:
net = nn.Sequential()
net.add_module('resnet_block_1', resnet_block(in_channels=1, out_channels=32, num_residuals=9, stride=1))
net.add_module('resnet_block_2', resnet_block(in_channels=32, out_channels=64, num_residuals=1, stride=2))
net.add_module('resnet_block_3', resnet_block(in_channels=64, out_channels=32, num_residuals=1, stride=2))
net.add_module('resnet_block_4', resnet_block(in_channels=32, out_channels=16, num_residuals=1, stride=2))
net.add_module('flatten', nn.Flatten())
net.add_module('fc1', nn.Sequential(nn.Linear(in_features=9248, out_features=3076, bias=True), nn.ReLU()))
net.add_module('fc2', nn.Sequential(nn.Linear(in_features=3076, out_features=796, bias=True), nn.ReLU()))
net.add_module('fc3', nn.Sequential(nn.Linear(in_features=796, out_features=199, bias=True), nn.ReLU()))
net.add_module('fc4', nn.Linear(in_features=199, out_features=2, bias=True))

In [ ]:
net

In [ ]:
## test
x = torch.rand(1,1,4618)
net(x)

In [ ]:
#net.load_state_dict(torch.load(runs_path+'CNN-Classification-DR16Q.pt'))

## Training

In [ ]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    net.cuda()
    device = torch.device('cuda')
    print ('USE GPU')
else:
    device = torch.device('cpu')
    print ('USE CPU')

USE GPU


In [ ]:
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()

In [ ]:
loss = nn.CrossEntropyLoss() # include softmax and cross entropy
optimizer = optim.AdamW(net.parameters(), lr=0.00003, betas=(0.9, 0.999), weight_decay=1e-4, amsgrad=False)

In [ ]:
num_epochs = 10
batch_size = 128
train_iter = DataLoader(TensorDataset(X_train, Y_train), batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
for epoch in range(num_epochs):
    net.train() 
    start = time.time()
    train_l_sum = 0.
    train_acc_sum = 0.
    for img, label in train_iter:
        if use_gpu:
            img, label =  img.to(device), label.to(device)
        optimizer.zero_grad()
        predict = net(img)
        l = loss(predict, label.view(-1))
        l.backward()
        optimizer.step()
        train_l_sum += l.data.item()
        train_acc_sum += accuracy(predict, label.view(-1))
    
    if ((epoch+1)%1 ==0):
        net.eval() 
        print('Epoch:',epoch, 'Acc:',train_acc_sum / len(train_iter), 'Loss:',train_l_sum / len(train_iter) )


Epoch: 0 Acc: 0.5884046052631579 Loss: 0.6638532986766413
Epoch: 1 Acc: 0.6959292763157895 Loss: 0.5634666587177076
Epoch: 2 Acc: 0.7386924342105263 Loss: 0.5028381449611563
Epoch: 3 Acc: 0.7592516447368421 Loss: 0.476529985666275
Epoch: 4 Acc: 0.7693256578947368 Loss: 0.460521988962826
Epoch: 5 Acc: 0.782483552631579 Loss: 0.44468317063231216
Epoch: 6 Acc: 0.7888569078947368 Loss: 0.4376163639520344
Epoch: 7 Acc: 0.7962582236842105 Loss: 0.42742776635446045
Epoch: 8 Acc: 0.7966694078947368 Loss: 0.4268218137715992
Epoch: 9 Acc: 0.813733552631579 Loss: 0.40741798987514094


In [ ]:
## save net
#torch.save(net.state_dict(), runs_path+'ResNet-Classification-DR16Q-Equally.pt')

In [ ]:
## Accuracy of each test group

net.eval()
test_iter = DataLoader(TensorDataset(X_test, Y_test), batch_size=128, shuffle=True, num_workers=4)

y_hats = np.array([])
ys = np.array([])
for img, label in test_iter:
    ys = np.concatenate((ys,label.view(-1).numpy()))
    if use_gpu:
        img, label =  img.to(device), label.to(device)
    y_hat = net(img)
    y_hats = np.concatenate((y_hats,y_hat.cpu().argmax(dim=1).detach().numpy()))
    

y_hat_0 = np.array([True if item == 0 else False for item in y_hats])
y_0 = np.array([True if item == 0 else False for item in ys])

y_hat_1 = np.array([True if item == 1 else False for item in y_hats])
y_1 = np.array([True if item == 1 else False for item in ys])

In [ ]:
print('galaxy:', np.sum(y_hat_0 & y_0)/np.sum(y_0))
print('quasar:', np.sum(y_hat_1 & y_1)/np.sum(y_1))